Using the Titanic dataset

    Use the function defined in acquire.py to load the Titanic data.

    Use the function defined in prepare.py to prepare the titanic data.

    Encode the categorical columns on train dataset. Create dummy variables of the categorical columns and concatenate them onto the dataframe. Remove the columns they are replacing. Repeat on validate and test.

    Create a function named preprocess_titanic that accepts the train, validate, and test titanic data, and returns the dataframes ready for modeling.

Using the Telco dataset

    Use the function defined in acquire.py to load the Telco data.

    Use the function defined in prepare.py to prepare the Telco data.

    Encode the categorical columns on train.
        Encode at least one column using .replace
        Encode at least one column using .map
        Encode the rest of the columns by creating dummy variables and concatenating them onto the dataframe.

    Repeat the same steps on validate and test.

    Create a function named prep_telco that accepts the train, validate, and test telco data, and returns the dataframes ready for modeling.

In [1]:
# step 1 for  both data sets:
# use acquire and prep that we already had developed

In [2]:
# imports:
import numpy as np
import pandas as pd
# custom modules:
import acquire as a
import prepare as p

In [3]:
# p.splitting_data?

In [4]:
# p.splitting_data(
#     p.clean_titanic(a.get_titanic_data()),
#     'survived')

In [5]:
# grab, prepare, and split our data sets
titanic_train, \
titanic_val, \
titanic_test = p.splitting_data(
    p.clean_titanic(a.get_titanic_data()),
    'survived')

this file exists, reading csv


In [6]:
# do the same thing for telco
telco_train, \
telco_val, \
telco_test = p.splitting_data(
    p.prep_telco(a.get_telco_data()),
    'churn')

this file exists, reading csv


In [7]:
telco_train.shape, telco_val.shape, telco_test.shape

((4225, 21), (1409, 21), (1409, 21))

In [8]:
titanic_train.shape, titanic_val.shape, titanic_test.shape

((534, 9), (178, 9), (179, 9))

In [9]:
# titanic preprocessing:
# we saw previously that the things that we need to encode are:
# sex (nominal, binary)
# and embark_town (nominal, multicategorical/nonbinary)

In [10]:
# class vs pclass

In [11]:
# get the encoded versions of sex and embark_town:

In [12]:
titanic_train.pclass.astype(int)

776    3
829    1
215    1
258    1
129    3
      ..
125    3
360    3
55     1
298    1
768    3
Name: pclass, Length: 534, dtype: int64

In [13]:
titanic_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 534 entries, 776 to 768
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   passenger_id  534 non-null    int64  
 1   survived      534 non-null    int64  
 2   pclass        534 non-null    object 
 3   sex           534 non-null    object 
 4   sibsp         534 non-null    int64  
 5   parch         534 non-null    int64  
 6   fare          534 non-null    float64
 7   embark_town   534 non-null    object 
 8   alone         534 non-null    int64  
dtypes: float64(1), int64(5), object(3)
memory usage: 41.7+ KB


In [14]:
# tidy up the pclass and drop passenger_id:
titanic_train.drop(columns='passenger_id')
titanic_train['pclass'] = titanic_train['pclass'].astype(int)

In [15]:
# encoding sex and embark_town, 
# the only object types we have left:

In [16]:
# lets assign the output of pd.get_dummies into a separate df:
titanic_train_encoded_cats = pd.get_dummies(titanic_train[['embark_town', 'sex']],
              drop_first=True).astype(int)

In [17]:
# glue that back into my original df:
titanic_preprocessed = pd.concat(
    [titanic_train,
    titanic_train_encoded_cats],
    axis=1).drop(columns=['sex', 'embark_town'])

In [18]:
titanic_preprocessed.head()

,passenger_id,survived,pclass,sibsp,parch,fare,alone,embark_town_Queenstown,embark_town_Southampton,sex_male
776,776,0,3,0,0,7.7500,1,1,0,1
829,829,1,1,0,0,80.0000,1,0,1,0
215,215,1,1,1,0,113.2750,0,0,0,0
258,258,1,1,0,0,512.3292,1,0,0,0
129,129,0,3,0,0,6.9750,1,0,1,1


In [19]:
def preprocess_titanic(train_df, val_df, test_df):
    '''
    preprocess_titanic will take in three pandas dataframes
    of our titanic data, expected as cleaned versions of this 
    titanic data set (see documentation on acquire.py and prepare.py)
    
    output:
    encoded, ML-ready versions of our clean data, with 
    columns sex and embark_town encoded in the one-hot fashion
    return: (pd.DataFrame, pd.DataFrame, pd.DataFrame)
    '''
    # with a looping structure:
    # for df in [train_df, val_df, test_df]:
    #     df.drop(blah blah blah)
    #     df['pclass'] = df['pclass'].astype(int)
    train_df = train_df.drop(columns='passenger_id')
    train_df['pclass'] = train_df['pclass'].astype(int)
    val_df = val_df.drop(columns='passenger_id')
    val_df['pclass'] = val_df['pclass'].astype(int)
    test_df = test_df.drop(columns='passenger_id')
    test_df['pclass'] = test_df['pclass'].astype(int)
    encoding_var = ['sex', 'embark_town']
    encoded_dfs = []
    for df in [train_df, val_df, test_df]:
        df_encoded_cats = pd.get_dummies(
            df[['embark_town', 'sex']],
              drop_first=True).astype(int)
        encoded_dfs.append(pd.concat(
            [df,
            df_encoded_cats],
            axis=1).drop(columns=['sex', 'embark_town']))
    return encoded_dfs
    

In [20]:
ttrain_encoded, tval_encoded, ttest_encoded = preprocess_titanic(
    titanic_train, 
    titanic_val, 
    titanic_test)

In [21]:
ttrain_encoded.head()

,survived,pclass,sibsp,parch,fare,alone,embark_town_Queenstown,embark_town_Southampton,sex_male
776,0,3,0,0,7.7500,1,1,0,1
829,1,1,0,0,80.0000,1,0,1,0
215,1,1,1,0,113.2750,0,0,0,0
258,1,1,0,0,512.3292,1,0,0,0
129,0,3,0,0,6.9750,1,0,1,1


In [22]:
# grabbing info:
# for exploration:
#  splitting_data(clean_titanic(acquire_titanic()))
# for modeling:
#  preprocess_titanic(splitting_data(clean_titanic(acquire_titanic())))

> for telco df:

In [23]:
telco_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4225 entries, 2865 to 5354
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   customer_id            4225 non-null   object 
 1   gender                 4225 non-null   object 
 2   senior_citizen         4225 non-null   int64  
 3   partner                4225 non-null   object 
 4   dependents             4225 non-null   object 
 5   tenure                 4225 non-null   int64  
 6   phone_service          4225 non-null   object 
 7   multiple_lines         4225 non-null   object 
 8   online_security        4225 non-null   object 
 9   online_backup          4225 non-null   object 
 10  device_protection      4225 non-null   object 
 11  tech_support           4225 non-null   object 
 12  streaming_tv           4225 non-null   object 
 13  streaming_movies       4225 non-null   object 
 14  paperless_billing      4225 non-null   object 
 15  monthl

In [24]:
# select dtype:
telco_train.select_dtypes('O')

,customer_id,gender,partner,dependents,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,total_charges,churn,contract_type,internet_service_type,payment_type
2865,4083-BFNYK,Female,Yes,No,Yes,Yes,Yes,Yes,No,No,No,Yes,No,3591.25,No,One year,Fiber optic,Credit card (automatic)
4107,5804-LEPIM,Female,No,No,Yes,No,No,No,No,No,No,No,Yes,139.05,Yes,Month-to-month,Fiber optic,Electronic check
3453,4895-TMWIR,Male,Yes,No,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,214.75,Yes,Month-to-month,NaN,Mailed check
923,1342-JPNKI,Male,No,No,Yes,Yes,No,No,No,No,Yes,No,Yes,834.1,Yes,Month-to-month,Fiber optic,Bank transfer (automatic)
2370,3397-AVTKU,Male,No,No,Yes,No,No,Yes,No,No,No,Yes,Yes,2548.55,No,Two year,DSL,Electronic check
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1286,1915-IOFGU,Female,No,No,Yes,No,No,No,No,No,No,No,No,70.5,Yes,Month-to-month,Fiber optic,Electronic check
4999,7025-WCBNE,Male,No,No,Yes,Yes,No,Yes,No,Yes,No,No,No,2754,No,Two year,DSL,Bank transfer (automatic)
6892,9788-HNGUT,Male,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,8594.4,No,Two year,Fiber optic,Credit card (automatic)
3718,5229-PRWKT,Male,No,No,Yes,Yes,No,No,No,No,No,Yes,Yes,649.65,Yes,Month-to-month,Fiber optic,Electronic check


In [25]:
# checking fields for object type:
# initialize some empty lists:
cat_cols, num_cols = [], []
for col in telco_train.columns:
    if telco_train[col].dtype == 'O':
        cat_cols.append(col)
    else:
        num_cols.append(col)

In [26]:
cat_cols

['customer_id',
 'gender',
 'partner',
 'dependents',
 'phone_service',
 'multiple_lines',
 'online_security',
 'online_backup',
 'device_protection',
 'tech_support',
 'streaming_tv',
 'streaming_movies',
 'paperless_billing',
 'total_charges',
 'churn',
 'contract_type',
 'internet_service_type',
 'payment_type']

In [27]:
cat_cols.remove('customer_id')

In [28]:
cat_cols

['gender',
 'partner',
 'dependents',
 'phone_service',
 'multiple_lines',
 'online_security',
 'online_backup',
 'device_protection',
 'tech_support',
 'streaming_tv',
 'streaming_movies',
 'paperless_billing',
 'total_charges',
 'churn',
 'contract_type',
 'internet_service_type',
 'payment_type']

In [29]:
# requests:
# use map once,
# use replace once
# great for binary categories that are nominal: True/False switches

In [30]:
# pd.Series.replace?

In [31]:
telco_train.loc[:, 'is_female'] = telco_train['gender'].replace({'Female': 1, 'Male': 0})

In [32]:
telco_train.partner.unique()

array(['Yes', 'No'], dtype=object)

In [33]:
telco_train.loc[:, 'has_partner'] = telco_train['partner'].map({'Yes': 1, 'No': 0})

In [34]:
# telco_train = telco_train.set_index('customer_id')

In [35]:
# debugging for the total_charges that turns out was not already
# a number:

In [36]:
telco_train['total_charges'] = (telco_train.total_charges + '0').astype(float)

In [37]:
def preprocess_telco(train_df, val_df, test_df):
    '''
    preprocess_telco will take in three pandas dataframes
    of our telco data, expected as cleaned versions of this 
    telco data set (see documentation on acquire.py and prepare.py)
    
    output:
    encoded, ML-ready versions of our clean data, with 
    columns sex and embark_town encoded in the one-hot fashion
    return: (pd.DataFrame, pd.DataFrame, pd.DataFrame)
    '''
    # with a looping structure:
    # go through the three dfs, set the index to customer id
    for df in [train_df, val_df, test_df]:
        df = df.set_index('customer_id')
        df['total_charges'] = df['total_charges'].astype(float)
    # initialize an empty list to see what needs to be encoded:
    encoding_vars = []
    # loop through the columns to fill encoded_vars with appropriate
    # datatype field names
    for col in train_df.columns:
        if train_df[col].dtype == 'O':
            encoding_vars.append(col)
    encoding_vars.remove('customer_id')
    # initialize an empty list to hold our encoded dataframes:
    encoded_dfs = []
    for df in [train_df, val_df, test_df]:
        df_encoded_cats = pd.get_dummies(
            df[encoding_vars],
              drop_first=True).astype(int)
        encoded_dfs.append(pd.concat(
            [df,
            df_encoded_cats],
            axis=1).drop(columns=encoding_vars))
    return encoded_dfs

In [38]:
tlco_train_encoded, \
tlco_val_encoded, \
tlco_test_encoded = preprocess_telco(telco_train, telco_val, telco_test)

In [39]:
tlco_train_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4225 entries, 2865 to 5354
Data columns (total 33 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   customer_id                            4225 non-null   object 
 1   senior_citizen                         4225 non-null   int64  
 2   tenure                                 4225 non-null   int64  
 3   monthly_charges                        4225 non-null   float64
 4   total_charges                          4225 non-null   float64
 5   is_female                              4225 non-null   int64  
 6   has_partner                            4225 non-null   int64  
 7   gender_Male                            4225 non-null   int64  
 8   partner_Yes                            4225 non-null   int64  
 9   dependents_Yes                         4225 non-null   int64  
 10  phone_service_Yes                      4225 non-null   int64  
 11  multip

In [40]:
tlco_train_encoded.head(1).T

,2865
customer_id,4083-BFNYK
senior_citizen,1
tenure,38
monthly_charges,95.0
total_charges,3591.25
is_female,1
has_partner,1
gender_Male,0
partner_Yes,1
dependents_Yes,0
